In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
import random
import os
from newspaper import Article
# pdfminer.six
# import pdfminer
# import pdfminer.high_level
pd.set_option("max_colwidth", 400)
from pandarallel import pandarallel

In [2]:
headers = {
        'accept': '*/*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh,zh-TW;q=0.9,zh-CN;q=0.8,en;q=0.7',
        'content-length': '0',
        'content-type': 'text/plain',
        'origin': 'https://www.efe.com',
        'referer': 'https://www.efe.com/',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
    }


def extract_urls(url):
    time.sleep(2)
    try:
        coverpage = requests.get(url, headers = headers).content
        soup = BeautifulSoup(coverpage, 'html.parser')
        rows = soup.find_all('article', {'id': 'liItem'})
        rows = [str(i) for i in rows]
    except:
        rows = 'error'
    return rows

def extract_row(row):
    soup = BeautifulSoup(row, 'html.parser')

    link=  soup.a['href']
    if link.startswith('/'):
        link = 'https://www.rfi.fr' + link
    text = re.sub("^\s+|\s+$", "", soup.text, flags=re.UNICODE)
    text = re.sub("\n +(\n)*", "\n", text, flags=re.UNICODE).split('\n')
    if len(text) == 2:
        text = ['None'] + text
    tag, date, title = text[0],text[1],text[2]
     
    return pd.Series([tag, date, title, link])


def extract_text(url):
    time.sleep(1)
    article = Article(url, keep_article_html=False)
    article.download()
    article.parse()
    return article.text

In [3]:
prefix  = 'https://www.efe.com/efe/english/search/50000541'
pages =\
    ['?q=attack&p=%s&s=0'%i for i in range(1, 543)] +\
    ['?q=conflict&p=%s&s=0'%i for i in range(1, 385)] +\
    ['?q=crime&p=%s&s=0'%i for i in range(1, 214)] +\
    ['?q=protest&p=%s&s=0'%i for i in range(1, 403)] +\
    ['?q=massacre&p=%s&s=0'%i for i in range(1, 51)] +\
    ['?q=refugee&p=%s&s=0'%i for i in range(1, 102)] +\
    ['?q=terrorism&p=%s&s=0'%i for i in range(1, 191)] +\
    ['?q=politics&p=%s&s=0'%i for i in range(1, 139)]  # politics

df = pd.DataFrame(pages,  columns =['url'] )
df['tags']= df.url.apply(lambda x: x.split('&')[0][3:])
df.url = prefix + df.url
df

,url,tags
0,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack
1,https://www.efe.com/efe/english/search/50000541?q=attack&p=2&s=0,attack
2,https://www.efe.com/efe/english/search/50000541?q=attack&p=3&s=0,attack
3,https://www.efe.com/efe/english/search/50000541?q=attack&p=4&s=0,attack
4,https://www.efe.com/efe/english/search/50000541?q=attack&p=5&s=0,attack
...,...,...
2015,https://www.efe.com/efe/english/search/50000541?q=politics&p=134&s=0,politics
2016,https://www.efe.com/efe/english/search/50000541?q=politics&p=135&s=0,politics
2017,https://www.efe.com/efe/english/search/50000541?q=politics&p=136&s=0,politics
2018,https://www.efe.com/efe/english/search/50000541?q=politics&p=137&s=0,politics


In [4]:
pandarallel.initialize(nb_workers= 64, progress_bar=True)
df[['rows']] = df['url'].parallel_apply(extract_urls)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
df[df.rows.str.len()==0]

,url,tags,rows


In [6]:
# df = df[df.rows.str.len()!=0].reset_index(drop=True)
df

,url,tags,rows
0,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"[<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/biden-harris-denounce-hate-crimes-against-asians-after-atlanta-attack/50000260-4492598"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMj8hf-P-2boAspvm6pAp6XE1M8D3Xqbz-P-2bB..."
1,https://www.efe.com/efe/english/search/50000541?q=attack&p=2&s=0,attack,"[<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/moscow-doubles-down-on-arrests-of-peaceful-pro-navalny-protesters/50000262-4454618"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMiZOrcacaJzx1sBj2e-P-2bTtaf3Xqbz-P-2bBR-P-2fn..."
2,https://www.efe.com/efe/english/search/50000541?q=attack&p=3&s=0,attack,"[<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/5th-afghan-journalist-killed-in-2-months/50000260-4431530"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMjxqdnZmjUOU9XDJ2g1cbzn3Xqbz-P-2bBR-P-2fnTP5WBQdtrSFQ2lznWY1aM4p4uc..."
3,https://www.efe.com/efe/english/search/50000541?q=attack&p=4&s=0,attack,"[<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/at-least-8-dead-31-injured-in-rocket-attack-on-center-of-kabul/50000262-4400203"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMhgRilKbdjC3CXwAoJojTDu3Xqbz-P-2bBR-P-2fnTP5WBQd..."
4,https://www.efe.com/efe/english/search/50000541?q=attack&p=5&s=0,attack,"[<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/at-least-6-injured-as-gunmen-storm-kabul-university/50000260-4383745"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMgNMYBgqHjv0PxiVBGl8O1G3Xqbz-P-2bBR-P-2fnTP5WBQdtrSFQ2lz..."
...,...,...,...
2015,https://www.efe.com/efe/english/search/50000541?q=politics&p=134&s=0,politics,"[<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/manlio-beltrones-elected-chairman-of-mexico-s-governing-pri/50000262-2690866"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?lVW2oAh2vjNTpdLHuVtkk1sdYO-P-2fiwR793Xqbz-P-2bBR-P-2fnTP5WBQdtrS..."
2016,https://www.efe.com/efe/english/search/50000541?q=politics&p=135&s=0,politics,"[<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/white-supremacist-cited-by-charleston-suspect-donated-money-to-republicans/50000262-2646139"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?lVW2oAh2vjOfpDLnizDR6KZc7Xqo-P-2bPmH3Xqbz-P-2bBR-..."
2017,https://www.efe.com/efe/english/search/50000541?q=politics&p=136&s=0,politics,"[<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/independent-candidate-in-puerto-rico-counts-on-social-networks/50000262-2609743"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?lVW2oAh2vjOcKOptHwa-P-2b87tI0Yphp1253Xqbz-P-2bBR

In [7]:
df = df.explode('rows', ignore_index=True)
df

,url,tags,rows
0,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/biden-harris-denounce-hate-crimes-against-asians-after-atlanta-attack/50000260-4492598"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMj8hf-P-2boAspvm6pAp6XE1M8D3Xqbz-P-2bBR..."
1,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/at-least-8-killed-54-injured-in-car-bombing-afghanistan/50000262-4487037"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMjwCkGhtuWG-P-2fzSxzMf2yW0t3Xqbz-P-2bBR-P-2fnTP5WBQdtrSF..."
2,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/hundreds-of-abducted-schoolgirls-freed-in-nigeria/50000260-4478132"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMh4YO5oFiBghyn3p-P-2f-P-2bY4Ix53Xqbz-P-2bBR-P-2fnTP5WBQdtrS..."
3,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/myanmar-military-junta-annuls-november-poll-results/50000260-4475079"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMj2fimzpT6sHZsHyOAx-P-2bceu3Xqbz-P-2bBR-P-2fnTP5WBQdtrSFQ..."
4,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/opposition-leader-navalny-loses-appeal-against-imprisonment/50000260-4470023"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMitfgAXlsP7KO2GqZwpRcvF3Xqbz-P-2bBR-P-2fnTP5WBQdt..."
...,...,...,...
20161,https://www.efe.com/efe/english/search/50000541?q=politics&p=138&s=0,politics,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/corporate/50000271"" id=""link_seccion"">Corporate</a>\n<a class=""link"" href=""/efe/english/corporate/london-gallery-relates-abstract-art-to-politics-and-society/50000271-2514744"" id=""link"">\n<img alt="""" class=""lazy vert"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?lVW2oAh2vjN6Lo8Aghi0R8IooXi9fp4CaRxt9Ac8dMC4YFZ2..."
20162,https://www.efe.com/efe/english/search/50000541?q=politics&p=138&s=0,politics,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/entertainment/50000264"" id=""link_seccion"">Entertainment</a>\n<a class=""link"" href=""/efe/english/entertainment/london-gallery-relates-abstract-art-to-politics-and-society/50000264-2512169"" id=""link"">\n<img alt="""" class=""lazy vert"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?lVW2oAh2vjN-P-2fX4gIjkovz8Jw3LLSe-P-..."
20163,https://www.efe.com/efe/english/search/50000541?q=politics&p=138&s=0,politics,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/democratic-rising-star-julian-castro-praises-hillary-clinton/50000262-2510104"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?lVW2oAh2vjOsTbIMIYgL7nKvGK2dIQs8aRxt9Ac8dMC4YFZ2

In [8]:
df.to_csv('efe_raw.csv', index=False)

In [3]:
df = pd.read_csv('efe_raw.csv', header=[0])
df

,url,tags,rows
0,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/biden-harris-denounce-hate-crimes-against-asians-after-atlanta-attack/50000260-4492598"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMj8hf-P-2boAspvm6pAp6XE1M8D3Xqbz-P-2bBR..."
1,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/at-least-8-killed-54-injured-in-car-bombing-afghanistan/50000262-4487037"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMjwCkGhtuWG-P-2fzSxzMf2yW0t3Xqbz-P-2bBR-P-2fnTP5WBQdtrSF..."
2,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/hundreds-of-abducted-schoolgirls-freed-in-nigeria/50000260-4478132"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMh4YO5oFiBghyn3p-P-2f-P-2bY4Ix53Xqbz-P-2bBR-P-2fnTP5WBQdtrS..."
3,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/myanmar-military-junta-annuls-november-poll-results/50000260-4475079"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMj2fimzpT6sHZsHyOAx-P-2bceu3Xqbz-P-2bBR-P-2fnTP5WBQdtrSFQ..."
4,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/opposition-leader-navalny-loses-appeal-against-imprisonment/50000260-4470023"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMitfgAXlsP7KO2GqZwpRcvF3Xqbz-P-2bBR-P-2fnTP5WBQdt..."
...,...,...,...
20161,https://www.efe.com/efe/english/search/50000541?q=politics&p=138&s=0,politics,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/corporate/50000271"" id=""link_seccion"">Corporate</a>\n<a class=""link"" href=""/efe/english/corporate/london-gallery-relates-abstract-art-to-politics-and-society/50000271-2514744"" id=""link"">\n<img alt="""" class=""lazy vert"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?lVW2oAh2vjN6Lo8Aghi0R8IooXi9fp4CaRxt9Ac8dMC4YFZ2..."
20162,https://www.efe.com/efe/english/search/50000541?q=politics&p=138&s=0,politics,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/entertainment/50000264"" id=""link_seccion"">Entertainment</a>\n<a class=""link"" href=""/efe/english/entertainment/london-gallery-relates-abstract-art-to-politics-and-society/50000264-2512169"" id=""link"">\n<img alt="""" class=""lazy vert"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?lVW2oAh2vjN-P-2fX4gIjkovz8Jw3LLSe-P-..."
20163,https://www.efe.com/efe/english/search/50000541?q=politics&p=138&s=0,politics,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/democratic-rising-star-julian-castro-praises-hillary-clinton/50000262-2510104"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?lVW2oAh2vjOsTbIMIYgL7nKvGK2dIQs8aRxt9Ac8dMC4YFZ2

In [4]:
# df.url.str.split('/').apply(lambda x: x[5]).value_counts()
df.tags.value_counts()

attack       5412
protest      4020
conflict     3829
crime        2130
terrorism    1894
politics     1378
refugee      1005
massacre      498
Name: tags, dtype: int64

In [5]:
# def extract_row(row):
#     soup = BeautifulSoup(row, 'html.parser')

#     link=  soup.a['href']
#     if link.startswith('/'):
#         link = 'https://www.rfi.fr' + link
#     text = re.sub("^\s+|\s+$", "", soup.text, flags=re.UNICODE)
#     text = re.sub("\n +(\n)*", "\n", text, flags=re.UNICODE).split('\n')
#     if len(text) == 2:
#         text = ['None'] + text
#     tag, date, title = text[0],text[1],text[2]
     
#     return pd.Series([tag, date, title, link])

def extract_row(row):
    soup = BeautifulSoup(row, 'html.parser')
    try:
        a = soup.find_all('a', id = 'link')[0]
        link = a['href']
        if link.startswith('/'):
            link = 'https://www.efe.com/' + link
#         topic = a.span.text

        h3 = a.h3.get_text('\n')
        h3 = re.sub("^\s+|\s+$", "", h3, flags=re.UNICODE)
        topic, title = h3.split('\n')[0], h3.split('\n')[1]
        
        abstract = a.find_all(class_ = 'entry')[0].text
        date = a.find_all(class_ = 'fecha')[0].text
     
        return pd.Series([topic, date, title, abstract, link])
    except:
        return pd.Series(['error']*5)

In [6]:
pandarallel.initialize(progress_bar=True)
df[['topic', 'date', 'title', 'abstract', 'link']] = df['rows'].parallel_apply(extract_row)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
df.head()

,url,tags,rows,topic,date,title,abstract,link
0,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/biden-harris-denounce-hate-crimes-against-asians-after-atlanta-attack/50000260-4492598"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMj8hf-P-2boAspvm6pAp6XE1M8D3Xqbz-P-2bBR...",US SHOOTING,20 Mar 2021,"Biden, Harris denounce hate crimes against Asians after Atlanta attack","United States President Joe Biden and Vice President Kamala Harris Friday condemned rising hate crimes in the American society during a visit to Atlanta, days after a white gunman killed eight people, most of them Asian-American women.",https://www.efe.com//efe/english/portada/biden-harris-denounce-hate-crimes-against-asians-after-atlanta-attack/50000260-4492598
1,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/world/50000262"" id=""link_seccion"">World</a>\n<a class=""link"" href=""/efe/english/world/at-least-8-killed-54-injured-in-car-bombing-afghanistan/50000262-4487037"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMjwCkGhtuWG-P-2fzSxzMf2yW0t3Xqbz-P-2bBR-P-2fnTP5WBQdtrSF...",AFGHANISTAN ATTACK,13 Mar 2021,"At least 8 killed, 54 injured in car bombing in Afghanistan","At least eight people were killed and 54 injured in a car bomb explosion close to a police check-post and civilian houses in Afghanistan's eastern Herat province, authorities said on Saturday.",https://www.efe.com//efe/english/world/at-least-8-killed-54-injured-in-car-bombing-afghanistan/50000262-4487037
2,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/hundreds-of-abducted-schoolgirls-freed-in-nigeria/50000260-4478132"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMh4YO5oFiBghyn3p-P-2f-P-2bY4Ix53Xqbz-P-2bBR-P-2fnTP5WBQdtrS...",NIGERIA KIDNAPPING,2 Mar 2021,Hundreds of abducted schoolgirls freed in Nigeria,"Nearly 300 girls who were kidnapped last Friday at a public school in the state of Zamfara, in northwestern Nigeria, were released Tuesday, according to the governor of the region, Bello Muhammad Matawalle.",https://www.efe.com//efe/english/portada/hundreds-of-abducted-schoolgirls-freed-in-nigeria/50000260-4478132
3,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/myanmar-military-junta-annuls-november-poll-results/50000260-4475079"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/recursos2/imagen.aspx?-P-2fL4Jfo8HOMj2fimzpT6sHZsHyOAx-P-2bceu3Xqbz-P-2bBR-P-2fnTP5WBQdtrSFQ...",MYANMAR COUP,26 Feb 2021,Myanmar military junta annuls November poll results,Myanmar's military junta-appointed poll body on Friday invalidated the results of general elections held in November.,https://www.efe.com//efe/english/portada/myanmar-military-junta-annuls-november-poll-results/50000260-4475079
4,https://www.efe.com/efe/english/search/50000541?q=attack&p=1&s=0,attack,"<article id=""liItem"">\n<a class=""link-seccion"" href=""/efe/english/portada/50000260"" id=""link_seccion"">Main</a>\n<a class=""link"" href=""/efe/english/portada/opposition-leader-navalny-loses-appeal-against-imprisonment/50000260-4470023"" id=""link"">\n<img alt="""" class=""lazy"" data-original=""//estaticos.efe.com/efecom/rec

In [8]:
df = df[~df.duplicated(subset='link')].reset_index(drop=True)
df = df.drop(columns=['url','rows'])
df

,tags,topic,date,title,abstract,link
0,attack,US SHOOTING,20 Mar 2021,"Biden, Harris denounce hate crimes against Asians after Atlanta attack","United States President Joe Biden and Vice President Kamala Harris Friday condemned rising hate crimes in the American society during a visit to Atlanta, days after a white gunman killed eight people, most of them Asian-American women.",https://www.efe.com//efe/english/portada/biden-harris-denounce-hate-crimes-against-asians-after-atlanta-attack/50000260-4492598
1,attack,AFGHANISTAN ATTACK,13 Mar 2021,"At least 8 killed, 54 injured in car bombing in Afghanistan","At least eight people were killed and 54 injured in a car bomb explosion close to a police check-post and civilian houses in Afghanistan's eastern Herat province, authorities said on Saturday.",https://www.efe.com//efe/english/world/at-least-8-killed-54-injured-in-car-bombing-afghanistan/50000262-4487037
2,attack,NIGERIA KIDNAPPING,2 Mar 2021,Hundreds of abducted schoolgirls freed in Nigeria,"Nearly 300 girls who were kidnapped last Friday at a public school in the state of Zamfara, in northwestern Nigeria, were released Tuesday, according to the governor of the region, Bello Muhammad Matawalle.",https://www.efe.com//efe/english/portada/hundreds-of-abducted-schoolgirls-freed-in-nigeria/50000260-4478132
3,attack,MYANMAR COUP,26 Feb 2021,Myanmar military junta annuls November poll results,Myanmar's military junta-appointed poll body on Friday invalidated the results of general elections held in November.,https://www.efe.com//efe/english/portada/myanmar-military-junta-annuls-november-poll-results/50000260-4475079
4,attack,RUSSIA NAVALNY,20 Feb 2021,Opposition leader Navalny loses appeal against imprisonment,A Moscow court on Saturday rejected an appeal from Russian opposition leader Alexei Navalny against his 3.5 year imprisonment after he was found guilty of violating the conditions of his parole.,https://www.efe.com//efe/english/portada/opposition-leader-navalny-loses-appeal-against-imprisonment/50000260-4470023
...,...,...,...,...,...,...
16054,politics,BRITAIN ART,19 Jan 2015,London gallery relates abstract art to politics and society,"""Black Square"" painted by Russian artist Kazimir Malevich in 1915, revolutionized the world of art. It is one of the prominent artifacts now in display in London's Whitechapel Gallery, which explores the relationship between abstract art and politics and society.",https://www.efe.com//efe/english/corporate/london-gallery-relates-abstract-art-to-politics-and-society/50000271-2514744
16055,politics,BRITAIN ART,16 Jan 2015,London gallery relates abstract art to politics and society,"""Black Square"" painted by Russian artist Kazimir Malevich in 1915, revolutionized the world of art. It is one of the prominent artifacts now in display in London's Whitechapel Gallery, which explores the relationship between abstract art and politics and society.",https://www.efe.com//efe/english/entertainment/london-gallery-relates-abstract-art-to-politics-and-society/50000264-2512169
16056,politics,US POLITICS,13 Jan 2015,Democratic rising star Julian Castro praises Hillary Clinton,"Housing and Urban Development Secretary Julian Castro, seen as one of the Democratic Party's rising stars, said here Tuesday that former Secretary of State Hillary Clinton ""would do enormous good"" for the United States if she became president.",https://www.efe.com//efe/english/world/democratic-rising-star-julian-castro-praises-hillary-clinton/50000262-2510104
16057,politics,MEXICO POLITICS,11 Jan 2015,PAN chairman says his party will regain Mexico's presidency,"The chairman of the opposition National Action Party, or PAN, said during a rally that the party would regain the presidency because it was growing and ""is ready"" to set the course for Mexico's future.",https://www.efe.com//efe/english/world/pan-chairman-says-his-party-will-regain-mexico-s-presidency/50000262-2508188


In [3]:
def extract_text(url):
    try:
        time.sleep(1)
        coverpage = requests.get(url, headers = headers).content
    except:
        print('error url', url)
        return 'error url'
    
    soup = BeautifulSoup(coverpage, 'html.parser')
    body1 = soup.find_all('div', id = 'div_texto')
    body2 = soup.find_all('li', class_ = 'content dont-break-out selected')
    body = max([body1, body2],key=len)
    
    if len(body):
        text = body[0].find_all('p')
        text = [p.text for p in text]
        text = '\n'.join(text)
    else:
        text = ''
        
    if len(text)==0:
        time.sleep(1)
        article = Article(url)
        article.download()
        article.parse()
        text = article.text
        if len(text)==0:
            print('error text', url)
            text = 'error text'
    return text

In [4]:
df = pd.read_csv('df_temp.csv', header=[0])
df

,tags,topic,date,title,abstract,link,text
0,attack,US SHOOTING,20 Mar 2021,"Biden, Harris denounce hate crimes against Asians after Atlanta attack","United States President Joe Biden and Vice President Kamala Harris Friday condemned rising hate crimes in the American society during a visit to Atlanta, days after a white gunman killed eight people, most of them Asian-American women.",https://www.efe.com//efe/english/portada/biden-harris-denounce-hate-crimes-against-asians-after-atlanta-attack/50000260-4492598,"United States President Joe Biden and Vice President Kamala Harris Friday condemned rising hate crimes in the American society during a visit to Atlanta, days after a white gunman killed eight people, most of them Asian-American women. \n\nIn his address from Emory University in Atlanta after meeting with Asian-American state legislators, Biden said the nation could no longer remain silent in ..."
1,attack,AFGHANISTAN ATTACK,13 Mar 2021,"At least 8 killed, 54 injured in car bombing in Afghanistan","At least eight people were killed and 54 injured in a car bomb explosion close to a police check-post and civilian houses in Afghanistan's eastern Herat province, authorities said on Saturday.",https://www.efe.com//efe/english/world/at-least-8-killed-54-injured-in-car-bombing-afghanistan/50000262-4487037,"At least eight people were killed and 54 injured in a car bomb explosion close to a police check-post and civilian houses in Afghanistan's eastern Herat province, authorities said on Saturday.\n\nThe heavy explosion tool place around 9.20 pm on Friday in the police district-14, an area populated by civilians, a military official from the district told EFE on the condition of anonymity. EFE-EPA"
2,attack,NIGERIA KIDNAPPING,2 Mar 2021,Hundreds of abducted schoolgirls freed in Nigeria,"Nearly 300 girls who were kidnapped last Friday at a public school in the state of Zamfara, in northwestern Nigeria, were released Tuesday, according to the governor of the region, Bello Muhammad Matawalle.",https://www.efe.com//efe/english/portada/hundreds-of-abducted-schoolgirls-freed-in-nigeria/50000260-4478132,"Nearly 300 girls who were kidnapped last Friday at a public school in the state of Zamfara, in northwestern Nigeria, were released Tuesday, according to the governor of the region, Bello Muhammad Matawalle.\n\n\n\n""It gladdens my heart to announce the release of the abducted students of GGSS Jangebe from captivity. This follows the scaling of several hurdles laid against our efforts. I enjoin ..."
3,attack,MYANMAR COUP,26 Feb 2021,Myanmar military junta annuls November poll results,Myanmar's military junta-appointed poll body on Friday invalidated the results of general elections held in November.,https://www.efe.com//efe/english/portada/myanmar-military-junta-annuls-november-poll-results/50000260-4475079,"Myanmar's military junta-appointed poll body on Friday invalidated the results of general elections held in November.\n\nDeposed leader Aung San Suu Kyi's National League for Democracy party had won the elections by a landslide.\n\n“The National League for Democracy’s 2020 election result is no longer valid,” Election Commission chief Thein Soe said after a meeting with political parties, acco..."
4,attack,RUSSIA NAVALNY,20 Feb 2021,Opposition leader Navalny loses appeal against imprisonment,A Moscow court on Saturday rejected an appeal from Russian opposition leader Alexei Navalny against his 3.5 year imprisonment after he was found guilty of violating the conditions of his parole.,https://www.efe.com//efe/english/portada/opposition-leader-navalny-loses-appeal-against-imprisonment/50000260-4470023,"A Moscow court on Saturday rejected an appeal from Russian opposition leader Alexei Navalny against his 3.5 year imprisonment after he was found guilty of violating the conditions of his parole.\n\nNavalny was detained last month after returning from Germany, where he was receiving treatment from a alleged poisoning attack by Russian gover

In [5]:
df[df.text.isnull() | (df.text == 'error text')]

,tags,topic,date,title,abstract,link,text
1969,attack,SPAIN BEARS,25 May 2018,The wolf and the bear: two iconic animals with disparate reputations in Spain,"The wolf and the Cantabrian brown bear are perhaps two of Spain's most symbolic animals; and yet, the emotions they stir in the communities they affect could hardly be more opposing.",https://www.efe.com//efe/english/corporate/the-wolf-and-bear-two-iconic-animals-with-disparate-reputations-in-spain/50000271-3627927,error text
3893,attack,INT. WOMEN'S DAY,8 Mar 2017,Women all over Europe rally to celebrate International Women's Day,"Hundreds of thousands of people across Europe's main cities rallied on Wednesday to celebrate International Women's Day, a worldwide celebration meant to raise civil awareness of women's rights and fight against gender-based discrimination, cultural sexism, gender violence and sexual abuse.",https://www.efe.com//efe/english/eventos/women-all-over-europe-rally-to-celebrate-international-s-day/50000272-3201560,error text
4364,attack,JUDAISM JERUSALEM,11 Oct 2016,Thousands of penitent Jews atone for their sins at Jerusalem's Western Wall,Tens of thousands of Jews from around the world gathered on Tuesday morning at the sacred Western Wall in Jerusalem's Old City to say penitential prayers before the start of Yom Kippur _ the holiest day in the Hebrew calendar _ at sunset.,https://www.efe.com//efe/english/corporate/thousands-of-penitent-jews-atone-for-their-sins-at-jerusalem-s-western-wall/50000271-3065007,error text
4420,attack,ISRAEL PERES,29 Sep 2016,Arab responses to Peres' death reveal controversy surrounding his figure,"As with most prominent statesmen, the figure of former Israeli President Shimon Peres has not been exempt from criticism and controversy, especially by those who saw him as one of the main stalwarts of Israel's contentious security policies.",https://www.efe.com//efe/english/corporate/arab-responses-to-peres-death-reveal-controversy-surrounding-his-figure/50000271-3054258,error text
4523,attack,PORTUGAL HISTORY,28 Aug 2016,Almeida's walls relive Napoleonic siege in Peninsular War,Around 400 reenactors gathered on Sunday in the Portuguese town of Almeida to recreate the fortified village's siege by Napoleonic troops in 1810.,https://www.efe.com//efe/english/corporate/almeida-s-walls-relive-napoleonic-siege-in-peninsular-war/50000271-3025124,error text
...,...,...,...,...,...,...,...
16054,politics,BRITAIN ART,19 Jan 2015,London gallery relates abstract art to politics and society,"""Black Square"" painted by Russian artist Kazimir Malevich in 1915, revolutionized the world of art. It is one of the prominent artifacts now in display in London's Whitechapel Gallery, which explores the relationship between abstract art and politics and society.",https://www.efe.com//efe/english/corporate/london-gallery-relates-abstract-art-to-politics-and-society/50000271-2514744,NaN
16055,politics,BRITAIN ART,16 Jan 2015,London gallery relates abstract art to politics and society,"""Black Square"" painted by Russian artist Kazimir Malevich in 1915, revolutionized the world of art. It is one of the prominent artifacts now in display in London's Whitechapel Gallery, which explores the relationship between abstract art and politics and society.",https://www.efe.com//efe/english/entertainment/london-gallery-relates-abstract-art-to-politics-and-society/50000264-2512169,NaN
16056,politics,US POLITICS,13 Jan 2015,Democratic rising star Julian Castro praises Hillary Clinton,"Housing and Urban Development Secretary Julian Castro, seen as one of the Democratic Party's rising stars, said here Tuesday that former Secretary of State Hillary Clinton ""would do enormous good"" for the United States if she became president.",https://www.efe.com//efe/english/world/democratic-rising-star-julian-castro-praises-hillary-clinton/50000262-2510104,NaN
16057,politics,MEXICO POLITICS,11 Jan 2015,PAN chairman says his party will regain Mexico's presidency,"The chairman of t

In [6]:
df1 = df[df.text.isnull() | (df.text == 'error text')].copy().reset_index(drop=True)
df1= df1.iloc[:,5:]
df1

,link,text
0,https://www.efe.com//efe/english/corporate/the-wolf-and-bear-two-iconic-animals-with-disparate-reputations-in-spain/50000271-3627927,error text
1,https://www.efe.com//efe/english/eventos/women-all-over-europe-rally-to-celebrate-international-s-day/50000272-3201560,error text
2,https://www.efe.com//efe/english/corporate/thousands-of-penitent-jews-atone-for-their-sins-at-jerusalem-s-western-wall/50000271-3065007,error text
3,https://www.efe.com//efe/english/corporate/arab-responses-to-peres-death-reveal-controversy-surrounding-his-figure/50000271-3054258,error text
4,https://www.efe.com//efe/english/corporate/almeida-s-walls-relive-napoleonic-siege-in-peninsular-war/50000271-3025124,error text
...,...,...
5845,https://www.efe.com//efe/english/corporate/london-gallery-relates-abstract-art-to-politics-and-society/50000271-2514744,NaN
5846,https://www.efe.com//efe/english/entertainment/london-gallery-relates-abstract-art-to-politics-and-society/50000264-2512169,NaN
5847,https://www.efe.com//efe/english/world/democratic-rising-star-julian-castro-praises-hillary-clinton/50000262-2510104,NaN
5848,https://www.efe.com//efe/english/world/pan-chairman-says-his-party-will-regain-mexico-s-presidency/50000262-2508188,NaN


In [7]:
pandarallel.initialize(nb_workers=16, progress_bar=False)
batch_size = 512

start = time.time()
for i in range(0,df1.shape[0],batch_size):
    df1.loc[i:i+batch_size-1, 'text']=\
    df1.loc[i:i+batch_size-1, 'link'].parallel_apply(extract_text).values
    
    end = time.time()
    print('--- %4d --- %.1f'%(i, end-start))
    start = end

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
---    0 --- 47.5
error url https://www.efe.com//efe/english/life/teen-climate-change-activist-greta-thunberg-meets-pope-francis/50000263-3955046
---  512 --- 175.8
--- 1024 --- 50.2
--- 1536 --- 47.6
--- 2048 --- 50.2
--- 2560 --- 48.2
--- 3072 --- 46.1
--- 3584 --- 47.2
--- 4096 --- 46.0
--- 4608 --- 47.1
--- 5120 --- 47.4
--- 5632 --- 20.6


In [13]:
df.loc[df[df.text.isnull() | (df.text == 'error text')].index,  'text'] = df1.text.values

In [14]:
df.text.str.len().describe()

count    16059.000000
mean      1939.888598
std       1127.884939
min          9.000000
25%       1272.500000
50%       1781.000000
75%       2367.000000
max      27770.000000
Name: text, dtype: float64

In [20]:
df[df.text.str.len()<100]

,tags,topic,date,title,abstract,link,text
2450,attack,error,error,error,error,error,error url
7178,conflict,CROATIA GOVERNMENT,19 May 2017,Croatia's largest business group's collapse threatens government stability,The liabilities of Croatia's largest business group are threatening its bankruptcy with the potential loss of tens of thousands of jobs and even the stability of the country's governmentced.,https://www.efe.com//efe/english/business/croatia-s-largest-business-group-collapse-threatens-government-stability/50000265-3271424,error url
10808,protest,VATICAN CLIMATE CHANGE,17 Apr 2019,Teen climate change activist Greta Thunberg meets Pope Francis,Swedish teen climate change activist Greta Thunberg met Pope Francis on Wednesday during the weekly general audience in the Vatican.,https://www.efe.com//efe/english/life/teen-climate-change-activist-greta-thunberg-meets-pope-francis/50000263-3955046,error url


In [21]:
df = df[df.text.str.len()>100]

In [22]:
df[df.duplicated(subset='text', keep=False)].sort_values('text')

,tags,topic,date,title,abstract,link,text
6810,conflict,SYRIA CONFLICT,8 Sep 2017,Russia announces death of IS war minister in bombing,"A Russian air strike killed the so-called ""war minister"" of the Islamic State terror organization near the Syrian city of Deir al-Zour, the Russian Defense Ministry announced on Friday.",https://www.efe.com//efe/english/world/russia-announces-death-of-is-war-minister-in-bombing/50000262-3373239,"A Russian air strike killed the so-called ""war minister"" of the Islamic State terror organization near the Syrian city of Deir al-Zour, the Russian Defense Ministry announced on Friday.\nThe raid destroyed an IS communications center and an underground command post where several IS warlords were gathered, the Defense Ministry statement said.\nThe meeting was attended by the war minister of the..."
6809,conflict,SYRIA CONFLICT,8 Sep 2017,Russia announces death of IS war minister in bombing,"A Russian air strike killed the so-called ""war minister"" of the Islamic State terror organization near the Syrian city of Deir al-Zour, the Russian Defense Ministry announced on Friday.",https://www.efe.com//efe/english/world/russia-announces-death-of-is-war-minister-in-bombing/50000262-3373267,"A Russian air strike killed the so-called ""war minister"" of the Islamic State terror organization near the Syrian city of Deir al-Zour, the Russian Defense Ministry announced on Friday.\nThe raid destroyed an IS communications center and an underground command post where several IS warlords were gathered, the Defense Ministry statement said.\nThe meeting was attended by the war minister of the..."
15798,politics,BRAZIL CORRUPTION,6 Jun 2017,Brazil's former house speaker arrested on corruption charges,A former speaker of Brazil's lower house of Congress who later served as tourism minister was detained Tuesday on suspicion of graft in connection with the 2014 soccer World Cup.,https://www.efe.com//efe/english/world/brazil-s-former-house-speaker-arrested-on-corruption-charges/50000262-3289094,"A former speaker of Brazil's lower house of Congress who later served as tourism minister was detained Tuesday on suspicion of graft in connection with the 2014 soccer World Cup.\nHenrique Eduardo Alves, who led the lower house from 2013-2015 before becoming tourism minister, was arrested at his home in Natal, capital of the northeastern state of Rio Grande do Norte.\nThe once-powerful politic..."
15797,politics,BRAZIL CORRUPTION,7 Jun 2017,Brazil's former house speaker arrested on corruption charges,A former speaker of Brazil's lower house of Congress who later served as tourism minister was detained Tuesday on suspicion of graft in connection with the 2014 soccer World Cup.,https://www.efe.com//efe/english/portada/brazil-s-former-house-speaker-arrested-on-corruption-charges/50000260-3289334,"A former speaker of Brazil's lower house of Congress who later served as tourism minister was detained Tuesday on suspicion of graft in connection with the 2014 soccer World Cup.\nHenrique Eduardo Alves, who led the lower house from 2013-2015 before becoming tourism minister, was arrested at his home in Natal, capital of the northeastern state of Rio Grande do Norte.\nThe once-powerful politic..."
13160,protest,BRAZIL RAPE/PROTEST,8 Jun 2016,Brazil's most famous beach covered with panties in anti-rape protest,A total of 420 pairs of women's panties were spread out Monday during an anti-rape protest on Rio's emblematic Copacabana Beach to represent the number of rapes perpetrated every 72 hours in Brazil.,https://www.efe.com//efe/english/varios/brazil-s-most-famous-beach-covered-with-panties-in-anti-rape-protest/50000269-2950245,"A total of 420 pairs of women's panties were spread out Monday during an anti-rape protest on Rio's emblematic Copacabana Beach to represent the number of rapes perpetrated every 72 hours in Brazil.\nThe demonstration was organized by the NGO Rio da Paz, known for its flamboyant protests against all kinds of violence

In [23]:
df.shape

(16056, 7)

In [24]:
df = df[~df.duplicated(subset='text')]
df.shape

(16004, 7)

In [27]:
df[df.duplicated(subset=['title', 'date'],keep= False)].sort_values('title')

,tags,topic,date,title,abstract,link,text
105,attack,AFGHANISTAN ATTACK,9 Feb 2020,2 American soldiers killed in Afghanistan attack,"At least two American soldiers were killed and six wounded in an alleged internal attack carried out by someone wearing an Afghan soldier's uniform in Nangarhar province in east Afghanistan, the United States military said Sunday.",https://www.efe.com//efe/english/portada/2-american-soldiers-killed-in-afghanistan-attack/50000260-4169725,"At least two American soldiers were killed and six wounded in an alleged internal attack carried out by someone wearing an Afghan soldier's uniform in Nangarhar province in east Afghanistan, the United States military said Sunday.\n\nUS Forces Spokesperson Sonny Leggett said in a statement that the man opened fire on the combined US and Afghan force with a machine gun."
106,attack,AFGHANISTAN ATTACK (ADDS VIDEO),9 Feb 2020,2 American soldiers killed in Afghanistan attack,"At least two American soldiers were killed and six wounded in an alleged internal attack carried out by someone wearing an Afghan soldier's uniform in Nangarhar province in east Afghanistan, the United States military said Sunday.",https://www.efe.com//efe/english/portada/2-american-soldiers-killed-in-afghanistan-attack/50000260-4169510,"At least two American soldiers were killed and six wounded in an alleged internal attack carried out by someone wearing an Afghan soldier's uniform in Nangarhar province in east Afghanistan, the United States military said Sunday.\n\nUS Forces Spokesperson Sonny Leggett said in a statement that the man opened fire on the combined US and Afghan force with a machine gun.\n\nThe six injured soldi..."
11451,protest,USA SHOOTING,26 Aug 2018,4 Dead in Florida mall shooting,"Four people died in a shooting on Sunday in a restaurant at a well-known mall in Jacksonville, Florida, local media reported.",https://www.efe.com//efe/english/world/4-dead-in-florida-mall-shooting/50000262-3730409,"Four people died in a shooting on Sunday in a restaurant at a well-known mall in Jacksonville, Florida, local media reported.\nAccording to Channel News4Jax, 11 people were wounded by gunfire, of whom four have died."
11450,protest,USA SHOOTING,26 Aug 2018,4 Dead in Florida mall shooting,"Four people died in a shooting on Sunday in a restaurant at a well-known mall in Jacksonville, Florida, local media reported.",https://www.efe.com//efe/english/world/4-dead-in-florida-mall-shooting/50000262-3730458,"Four people died in a shooting on Sunday in a restaurant at a well-known mall in Jacksonville, Florida, local media reported.\nAccording to Channel News4Jax, 11 people were wounded by gunfire, of whom four have died.\nWhile local law enforcement authorities have released no specific figures, they are speaking of ""multiple"" fatalities at the Jacksonville Landing mall, including many people take..."
14664,terrorism,TRUMP IMMIGRATION,30 Jan 2017,Attorneys general of 16 states condemn Trump's ban on Muslims,The attorneys general of 16 US states on Sunday condemned President Donald Trump's temporary ban on granting entry to refugees and citizens from seven Muslim-majority nations.,https://www.efe.com//efe/english/world/attorneys-general-of-16-states-condemn-trump-s-ban-on-muslims/50000262-3163354,"The attorneys general of 16 US states on Sunday condemned President Donald Trump's temporary ban on granting entry to refugees and citizens from seven Muslim-majority nations.\nThe attorneys general issued a joint statement saying, among other things, ""We are committed to working to ensure that as few people as possible suffer from the chaotic situation that it has created.""\nThey added that t..."
12837,protest,TRUMP IMMIGRATION,30 Jan 2017,Attorneys general of 16 states condemn Trump's ban on Muslims,The attorneys general of 16 US states on Sunday condemned President Donald Trump's temporary ban on granting entry to refugees and citizens from seven Muslim-majority nations.,https://www.efe.com//efe

In [28]:
df = df[~df.duplicated(subset=['title', 'date'])]
df.shape

(15976, 7)

In [30]:
df[df.duplicated(subset=['abstract'],keep= False)].sort_values('abstract')

,tags,topic,date,title,abstract,link,text
16055,politics,BRITAIN ART,16 Jan 2015,London gallery relates abstract art to politics and society,"""Black Square"" painted by Russian artist Kazimir Malevich in 1915, revolutionized the world of art. It is one of the prominent artifacts now in display in London's Whitechapel Gallery, which explores the relationship between abstract art and politics and society.",https://www.efe.com//efe/english/entertainment/london-gallery-relates-abstract-art-to-politics-and-society/50000264-2512169,"""Black Square"" painted by Russian artist Kazimir Malevich in 1915, revolutionized the world of art. It is one of the prominent artifacts now in display in London's Whitechapel Gallery, which explores the relationship between abstract art and politics and society.\nThe gallery is currently highlighting ""Adventures of the Black Square: Abstract Art and Society 1915-2015"" an exhibition which open..."
16054,politics,BRITAIN ART,19 Jan 2015,London gallery relates abstract art to politics and society,"""Black Square"" painted by Russian artist Kazimir Malevich in 1915, revolutionized the world of art. It is one of the prominent artifacts now in display in London's Whitechapel Gallery, which explores the relationship between abstract art and politics and society.",https://www.efe.com//efe/english/corporate/london-gallery-relates-abstract-art-to-politics-and-society/50000271-2514744,"By Alejandro Salazar\n\nLondon, Jan 15 (EFE).- ""Black Square"" painted by Russian artist Kazimir Malevich in 1915, revolutionized the world of art. It is one of the prominent artifacts now in display in London's Whitechapel Gallery, which explores the relationship between abstract art and politics and society.\n\nThe gallery is currently highlighting ""Adventures of the Black Square: Abstract Ar..."
5027,attack,MEXICO MISSING/FILM,18 Oct 2015,Documentary film on Ayotzinapa case sparks controversy in Mexico,"A documentary film's trailer showing the incineration of dozens of Ayotzinapa students at a municipal waste dump has sparked controversy in Mexico, where protesters suspect the movie will uncritically present the government's version of what happened to those teacher trainees.",https://www.efe.com//efe/english/entertainment/documentary-film-on-ayotzinapa-case-sparks-controversy-in-mexico/50000264-2740420,"A documentary film's trailer showing the incineration of dozens of Ayotzinapa students at a municipal waste dump has sparked controversy in Mexico, where protesters suspect the movie will uncritically present the government's version of what happened to those teacher trainees.\nThe film, titled ""La noche de Iguala,"" premiered Friday at 30 theaters despite a petition by a group of citizens to b..."
5024,attack,MEXICO MISSING/FILM,20 Oct 2015,Documentary film on Ayotzinapa case sparks controversy in Mexico,"A documentary film's trailer showing the incineration of dozens of Ayotzinapa students at a municipal waste dump has sparked controversy in Mexico, where protesters suspect the movie will uncritically present the government's version of what happened to those teacher trainees.",https://www.efe.com//efe/english/corporate/documentary-film-on-ayotzinapa-case-sparks-controversy-in-mexico/50000271-2742610,"Mexico City, Oct 17 (EFE).- A documentary film's trailer showing the incineration of dozens of Ayotzinapa students at a municipal waste dump has sparked controversy in Mexico, where protesters suspect the movie will uncritically present the government's version of what happened to those teacher trainees.\n\nThe film, titled ""La noche de Iguala,"" premiered Friday at 30 theaters despite a petiti..."
1416,attack,VENEZUELA CRISIS,8 Oct 2018,Suspect in attack on Venezuela's Maduro commits suicide,"A member of the capital's city council arrested last week on suspicion he was involved in a failed attack against President Nicolas Maduro took his own life Monday, the Venezuelan attorney general said.",https://www.efe.com//efe/eng

In [37]:
df = df[~df.duplicated(subset=['abstract'],keep= 'last')]
df.shape

(15943, 7)

In [39]:
df = df.sort_values('date').reset_index(drop=True)
df

,tags,topic,date,title,abstract,link,text
0,conflict,YEMEN CONFLICT,1 Apr 2015,At least 37 dead in dairy factory bombing in Yemen,"At least 37 people were killed and another 80 wounded in an attack Tuesday night when the Saudi-led Arab coalition bombed a dairy factory in the eastern Yemeni city of al-Hodeidah, according to a statement from the Houthi rebels.",https://www.efe.com//efe/english/world/at-least-37-dead-in-dairy-factory-bombing-yemen/50000262-2576307,"At least 37 people were killed and another 80 wounded in an attack Tuesday night when the Saudi-led Arab coalition bombed a dairy factory in the eastern Yemeni city of al-Hodeidah, according to a statement from the Houthi rebels.\nWitnesses said the factory was located near military barracks and near the headquarters of the maritime defense forces, which were also bombed by the coalition aircr..."
1,refugee,JIHADISTS SYRIA,1 Apr 2015,IS seizes control of Palestinian refugee camp in Damascus,"The Islamic State took control on Wednesday of large parts of al-Yarmouk Palestinian refugee camp in the south of the Syrian capital, according to the Syrian Observatory for Human Rights, or SOHR.",https://www.efe.com//efe/english/world/is-seizes-control-of-palestinian-refugee-camp-in-damascus/50000262-2576507,"The Islamic State took control on Wednesday of large parts of al-Yarmouk Palestinian refugee camp in the south of the Syrian capital, according to the Syrian Observatory for Human Rights, or SOHR.\nAfter clashes with Aknaf Beit al-Maqdis, a Syrian Islamist rebel faction, IS militants seized areas to the southeast and southwest of al-Yarmouk, pushing to the center of the camp, five kilometers (..."
2,terrorism,ENTERTAINMENT FILM,1 Apr 2015,Stephanie Sigman hopes Bond girl gig will open doors in Hollywood,"Stephanie Sigman hopes her role as the first Mexican Bond girl in ""Spectre,"" the latest film in the James Bond series, will open doors in Hollywood.",https://www.efe.com//efe/english/agencia-efe/stephanie-sigman-hopes-bond-girl-gig-will-open-doors-in-hollywood/50000270-2576562,"Stephanie Sigman hopes her role as the first Mexican Bond girl in ""Spectre,"" the latest film in the James Bond series, will open doors in Hollywood.\n""I think there's going to be a big opening, I expect this project will open a higher, different door to another path,"" Sigman told Efe in Mexico City, where scenes are being shot for the latest 007 film.\nSigman said her role in 2011's ""Miss Bala..."
3,politics,FRANCE POLITICS,1 Apr 2015,"Sarkozy declared an ""assisted witness"" after testifying on campaign funds","Former French President Nicolas Sarkozy was declared on Wednesday an ""assisted witness"" by the public prosecutor of Paris after testifying for several hours regarding on the funding and account of his 2012 presidential campaign.",https://www.efe.com//efe/english/world/sarkozy-declared-an-assisted-witness-after-testifying-on-campaign-funds/50000262-2576475,"Former French President Nicolas Sarkozy was declared on Wednesday an ""assisted witness"" by the public prosecutor of Paris after testifying for several hours regarding on the funding and account of his 2012 presidential campaign.\nAn ""assisted witness"" is a legal designation in French law referring a witness against whom there is no formal investigation, but who may eventually be implicated by ..."
4,attack,TURKEY TERRORISM,1 Apr 2015,"Attack on Istanbul police station leaves 1 assailant dead, 2 police wounded","An attack on an Istanbul police station on Wednesday left of one of the two perpetrators dead and two policemen wounded, Turkish media reported.",https://www.efe.com//efe/english/world/attack-on-istanbul-police-station-leaves-1-assailant-dead-2-wounded/50000262-2576542,"An attack on an Istanbul police station on Wednesday left of one of the two perpetrators dead and two policemen wounded, Turkish media reported.\n""The assailants opened fire on police stationed at the entrance of the building in Istanbul's Fatih district 

In [ ]:
df.to_csv('efc_final.csv', index=False)

In [10]:
pandarallel.initialize(nb_workers=16, progress_bar=False)
batch_size = 512

df['text']= None

start = time.time()
for i in range(0,df.shape[0],batch_size):
    df.loc[i:i+batch_size-1, 'text']=\
    df.loc[i:i+batch_size-1, 'link'].parallel_apply(extract_text).values
    
    end = time.time()
    print('--- %4d --- %.1f'%(i, end-start))
    start = end

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
---    0 --- 43.7
---  512 --- 48.3
--- 1024 --- 54.3
error text https://www.efe.com//efe/english/corporate/the-wolf-and-bear-two-iconic-animals-with-disparate-reputations-in-spain/50000271-3627927
--- 1536 --- 49.2
error url error
--- 2048 --- 49.9
--- 2560 --- 48.1
--- 3072 --- 49.4
error text https://www.efe.com//efe/english/eventos/women-all-over-europe-rally-to-celebrate-international-s-day/50000272-3201560
--- 3584 --- 50.2
error text https://www.efe.com//efe/english/corporate/arab-responses-to-peres-death-reveal-controversy-surrounding-his-figure/50000271-3054258
error text https://www.efe.com//efe/english/corporate/almeida-s-walls-relive-napoleonic-siege-in-peninsular-war/50000271-3025124
error text https://www.efe.com//efe/english/corporate/thousands-of-penitent-jews-atone-for-their-sins-at-jerusalem-s-western-wall/50000271-3065007
--- 

Process ForkPoolWorker-364:


error urlerror urlerror urlerror url  error urlerror urlerror url   error urlhttps://www.efe.com//efe/english/portada/trump-calls-for-calm-as-hong-kong-protests-heat-up/50000260-4042768https://www.efe.com//efe/english/world/women-in-the-forefront-after-a-month-of-lebanon-protests/50000262-4113080https://www.efe.com//efe/english/portada/greta-thunberg-leads-environmental-protest-at-un/50000260-4053738    error url error url

Traceback (most recent call last):


error url
https://www.efe.com//efe/english/destacada/controversial-citizenship-law-comes-into-effect-in-india/50000261-4147919
https://www.efe.com//efe/english/world/beijing-backs-hong-kong-authorities-slams-local-external-forces/50000262-4002096https://www.efe.com//efe/english/world/striking-teachers-and-doctors-in-honduras-bring-protest-to-us-embassy/50000262-3990110 error urlhttps://www.efe.com//efe/english/portada/hong-kong-protesters-ramp-up-on-china-national-day/50000260-4076615https://www.efe.com//efe/english/portada/suu-kyi-arrives-for-landmark-rohingya-hearings-at-the-hague/50000260-4129758
https://www.efe.com//efe/english/portada/state-of-emergency-declared-in-santiago-de-chile-to-quell-metro-fare-protests/50000260-4090635

...

  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)



https://www.efe.com//efe/english/world/argentina-protests-expropriation-of-agribusiness-vicentin/50000262-4277093

  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)


KeyboardInterrupt


KeyboardInterrupt: 

In [38]:
df.to_csv('df_temp.csv', index = False)

In [13]:
df = df[df.text != 'error text'].reset_index(drop= True)
df

,tag,date,title,link,text
0,Karachi affair,04/03/2021,Former French Prime Minister Edouard Balladur acquitted of corruption,https://www.rfi.fr/en/france/20210304-former-french-prime-minister-edouard-balladur-acquitted-of-corruption-jacques-chirac-nicolas-sarkozy-karachi-politics,"\n French former Prime Minister Edouard Balladur has been acquitted of charges related to a campaign financing scandal dating back to the 1990s, in a verdict in a Paris court today. However, his former defence minister François Léotard was handed a two year suspended sentence and a fine.\n Edouard Balladur, Prime Minister under Jacques Chirac and now 91 years ..."
1,Coronavirus,28/02/2021,French public health chief hails efficacy of Covid curfew,https://www.rfi.fr/en/france/20210228-french-public-health-chief-hails-efficacy-of-covid-curfew-lockdown-last-resort,"\n France’s top public health official Jérôme Salomon has hailed the nationwide curfew imposed by the government as part of its effort to stem the spread of the coronavirus.\n ""We are in a critical period of a race against time,” Salomon told the Journal du Dimanche. “But in this race against time, we have gained time thanks to restrictions and curfews.”\nMore..."
2,France,27/02/2021,French PM urges police to enforce Covid measures in high alert zones,https://www.rfi.fr/en/france/20210227-castex-urges-police-to-enforce-anti-coronavirus-measures-more-strictly,"\n French Prime Minister Jean Castex has called on police chiefs in 20 departments under heightened surveillance threatened by an explosion of coronavirus cases to crack down on citizens flouting the rules aimed at stemming the spread of the illness.\n The disease, which has killed just over 86,000 people in France since January 2020, shows no signs of abating..."
3,France,11/02/2021,French far-right's Le Pen faces off with hardline interior minister in TV debate,https://www.rfi.fr/en/france/20210211-french-far-right-leader-marine-le-pen-faces-off-with-hardline-interior-minister-gerald-darmanin-in-tv-debate,"\n Marine Le Pen, the leader of France's far-right, and hardline Interior Minister Gérald Darmanin are to clash Thursday evening on public TV channel France 2 in a debate seen as litmus test for the 2022 presidential election. The main topics: Islamism, immigration and security.\n The debate comes as opinion polls show Le Pen is likely to face off again agains..."
4,Paris Perspective,11/02/2021,Paris Perspective #4: The future of populism and the French far right,https://www.rfi.fr/en/podcasts/paris-perspective/20210211-paris-perspective-4-the-future-of-populism-and-the-french-hard-right,"\n Launched by Jean-Marie Le Pen in 1972, the Front National became synonymous with anti-semitism, anti-immigrant policy and protectionism over five decades, drawing its support base from disenfranchised French voters with a rallying cry of ""The People First"". Sound familiar? In this edition of Paris Perspective, we take a look at the state of France’s far-right Rassemblemen..."
...,...,...,...,...,...
7098,China - Australia,21/03/2010,China-Australia tension over Rio Tinto spy claims,https://www.rfi.fr/en/asia-pacific/20100321-china-australia-tension-over-rio-tinto-spy-claims,"\n Four employees of the mining company Rio Tinto will go on trial in Shanghai on Monday, accused of industrial espionage. One of the defendants is an Australian national, and the case has raised bilateral tensions between China and Australia.\n The four defendants, one Australian, three Chinese, were arrested last July during negotiations about the price of i..."
7099,Nepal,20/03/2010,Nepalese peace broker Koirala dies,https://www.rfi.fr/en/asia-pacific/20100320-nepalese-peace-broker-koirala-dies,"\n Girija Prasad Koirala, the former Nepalese Prime Minister who brokered the 2006 peace deal that ended a 10-year conflict with Maoist rebels, died on Saturday aged 85. He had been suffering from respiratory disease and passed away in Kathmandu around midday. Thousands of people 

In [14]:
df.to_csv('rfi_more_final.csv', index=False)